# Model For Prediction

installing and importing yfinance and pandas to use those for our prediction and calculation

In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf

In [4]:
import pandas as pd

Processing all the datas from history

In [5]:

def processing_currencies(currenciestype):
    df=yf.download(currenciestype)
    
    df.columns=['open','high','low','close','adj.close','volume']
    
    last_value = df['close'].iloc[-1]
    last_value
    df['close'] = df['open'].shift(-1)
    
    df.loc[df.index[-1], 'close'] = last_value
    
    del df['adj.close']
    del df['volume']
    
    df=df.dropna()
    
    
    return df



df1=processing_currencies('EURUSD=X')
df2=processing_currencies('GBPUSD=X')
df3=processing_currencies('USAU')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Here optimizing time period so we cant get more accurate results and also experimenting with other variables to find the optimized solution


In [6]:
df3=df3.loc["2013-12-01":].copy()

df2=df2.loc["2000-12-01":].copy()

In [7]:
def add_columns(df):
    df['pricefluctuation'] = df['high'] - df['low']
    df['changes'] = df['close'] - df['open']
    df['change%'] = (df['changes'] / df['open']) * 100
    df['target'] = 0 
    df.loc[df['change%'] > 0, 'target'] = 1
    df.loc[df['change%'] < 0, 'target'] = 0
    df['target'] = df['target'].shift(-1)
    df.loc[df.index[-1], 'target'] = 0
    df['target']=df['target'].astype(int)
    return df

df1=add_columns(df1)
df2=add_columns(df2)
df3=add_columns(df3)
 

 Using Machine Learning with random forest classifier to make a predicting model

In [8]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [57]:
def process_data(df, horizons):
    new_predictors = []

    for horizon in horizons:
        rolling_averages = df.rolling(horizon).mean()

        ratio_column = f'Close_Ratio_{horizon}'
        df[ratio_column] = df['close'] / rolling_averages['close']

        trend_column = f'Trend_{horizon}'
        df[trend_column] = df.shift(1).rolling(horizon).sum()['target']

        new_predictors += [ratio_column, trend_column]

    return df, new_predictors
 
horizons = [2, 5, 60, 250, 1000]
df1, new_predictors1 = process_data(df1, horizons)

df2, new_predictors2 = process_data(df2, horizons)
 
df3, new_predictors3 = process_data(df3, horizons)


df2

,open,high,low,close,pricefluctuation,changes,change%,target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,,
2007-10-04,2.032603,2.042400,2.028315,2.038487,0.014085,0.005884,0.289466,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-10-05,2.038487,2.043820,2.031901,2.041900,0.011919,0.003413,0.167426,0,1.000836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-10-08,2.041900,2.042901,2.033595,2.035499,0.009306,-0.006401,-0.313462,1,0.998430,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-10-09,2.035499,2.038985,2.026301,2.037697,0.012684,0.002198,0.107994,1,1.000540,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-10-10,2.037697,2.047209,2.037697,2.041108,0.009511,0.003411,0.167374,0,1.000836,2.0,1.001064,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-12,1.231694,1.233076,1.220718,1.218398,0.012358,-0.013296,-1.079499,0,0.994573,1.0,0.994329,4.0,0.972087,24.0,0.986882,129.0,0.946112,492.0
2023-10-13,1.218398,1.222345,1.213268,1.215480,0.009076,-0.002918,-0.239456,1,0.998801,0.0,0.993037,3.0,0.970659,24.0,0.984342,128.0,0.943934,492.0
2023-10-16,1.215480,1.219959,1.214432,1.221255,0.005526,0.005774,0.475070,0,1.002370,1.0,0.998406,3.0,0.976050,25.0,0.988781,129.0,0.948497,493.0


In [10]:
df1=df1.dropna()
df2=df2.dropna()
df3=df3.dropna()
 

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=350, min_samples_split=200,random_state=1)

In [12]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train['target'])
    preds=model.predict_proba(test[predictors])[:,1]
    preds[preds>=.50]=1
    preds[preds<.50]=0
    preds=pd.Series(preds,index=test.index,name='Predictions')
    combined=pd.concat([test['target'],preds],axis=1)
    return combined

In [13]:
def backtest(data,model,predictors,start=2500,step=250):
    all_predictions=[]
    
    
    for i in range(start,data.shape[0],step):
        train=data.iloc[0:i].copy()
        test=data.iloc[i:(i+step)].copy()
        predictions=predict(train,test,predictors,model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)


def backtest1(data,model,predictors,start=250,step=25):
    all_predictions=[]
    
    
    for i in range(start,data.shape[0],step):
        train=data.iloc[0:i].copy()
        test=data.iloc[i:(i+step)].copy()
        predictions=predict(train,test,predictors,model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)


In [14]:
predictions1=backtest(df1,model,new_predictors1)
predictions2=backtest1(df2,model,new_predictors2)
predictions3=backtest1(df3,model,new_predictors3)


In [53]:
predictions1.tail(1)['Predictions']


Date
2023-10-18    0.0
Name: Predictions, dtype: float64

Here it gives prediction 

In [15]:
predictions1['Predictions'].value_counts()
predictions2['Predictions'].value_counts()
predictions3['Predictions'].value_counts()


0.0    1098
1.0     138
Name: Predictions, dtype: int64

In [41]:
from sklearn.metrics import precision_score
print(precision_score(predictions1['target'],predictions1['Predictions'], average='micro'))
print(precision_score(predictions2['target'],predictions2['Predictions'], average='micro'))
print(precision_score(predictions3['target'],predictions3['Predictions'], average='micro')
)

0.5183845690174804
0.5047181841366998
0.5477346278317152


In [17]:
df3.tail(1)

,open,high,low,close,pricefluctuation,changes,change%,target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,,
2023-10-17,3.59,3.77,3.44,3.48,0.33,-0.11,-3.064064,0,0.984441,1.0,1.039427,3.0,0.941346,23.0,0.823326,109.0,0.476898,443.0


Here predicted value is stored and based on this email is sent

In [29]:
print(df1.tail()['target'])
print(df2.tail()['target'])
print(df3.tail()['target'])

Date
2023-10-12    0
2023-10-13    1
2023-10-16    1
2023-10-17    0
2023-10-18    0
Name: target, dtype: int32
Date
2023-10-12    0
2023-10-13    1
2023-10-16    0
2023-10-17    0
2023-10-18    0
Name: target, dtype: int32
Date
2023-10-11    0
2023-10-12    1
2023-10-13    1
2023-10-16    0
2023-10-17    0
Name: target, dtype: int32


## Setting up email


All the process of sending email is done here

In [19]:
pip install email

  Using cached email-4.0.2.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [20]:
pip install ssl

  Using cached ssl-1.16.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [28 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\Users\Crossy\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\__init__.py", line 247, in <module>
      monkey.patch_all()
    File "C:\Users\Crossy\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\monkey.py", line 97, in patch_all
      patch_for_msvc_specialized_compiler()
    File "C:\Users\Crossy\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\monkey.py", line 157, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
                  ^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\Crossy\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\monkey.py", line 147, in patch_params
      mod = import_module(mod_n

In [21]:
#pip install smtplib

In [22]:
import ssl

In [23]:
import smtplib

In [24]:
from email.message import EmailMessage

In [25]:
email_sender=' '
email_password=' '
email_receiver=' '

Here the sender email, their password, and also reciever email is entered.
For privacy purpose I have removed mine, but it works

In [30]:
subject = 'Trade Alert'

It sends email

In [42]:
def send_mail(body):
    em=EmailMessage()
    em['From']=email_sender
    em['To']=email_receiver
    em['Subject']=subject
    em.set_content(body)
    context=ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com',465,context=context) as smtp:
        smtp.login(email_sender,email_password)
        smtp.sendmail(email_sender,email_receiver,em.as_string())

# USING CSV FILE FOR READ,WRITE, APPEND AND DELETE OF STOCK INFORMATION

In [32]:
import csv

Now here CSV file is used to store all the trades, which helps us for stop-loss/cut-profit strategis

In [33]:
csv_file_path='stock_details.csv'

existing_data = pd.read_csv(csv_file_path)

print(existing_data)

Empty DataFrame
Columns: [Date, Name, Price, Quantity, ID]
Index: []


In [34]:
last_date3 = df3.tail(1).index[0].date()
print("Date3:", last_date3)

last_date2 = df2.tail(1).index[0].date()
print("Date2:", last_date2)

last_date1 = df1.tail(1).index[0].date()
print("Date1:", last_date1)


Date3: 2023-10-17
Date2: 2023-10-18
Date1: 2023-10-18


In [60]:
target_value3 = predictions3.tail(1)['Predictions'][0].astype(int)
print("Target Value3:", target_value3)
      

target_value2 = predictions2.tail(1)['Predictions'][0].astype(int)
print("Target Value2:", target_value2)
      
target_value1 = predictions1.tail(1)['Predictions'][0].astype(int)
print("Target Value1:", target_value1)

open_price3 = df3.tail(1)["open"].values[0]
print("Open Price3:", open_price3)

open_price2 = df2.tail(1)["open"].values[0]
print("Open Price2:", open_price2)

open_price1 = df1.tail(1)["open"].values[0]
print("Open Price1:", open_price1)

Target Value3: 0
Target Value2: 0
Target Value1: 0
Open Price3: 3.5899999141693115
Open Price2: 1.2184275388717651
Open Price1: 1.0578652620315552


In [36]:
x1=0
x2=0
x3=0
y1=0
y2=0
y3=0
z1=0
z2=0
z3=0

here it checks for the prediction and based on it, it sends email to buy that currencies

1 denotes buying!

In [52]:
existing_data = pd.read_csv(csv_file_path)    

if target_value1==1:
    new_id = len(existing_data) + 1
    duplicate_condition = (
        (existing_data["Date"] == last_date1) &
        (existing_data["Name"] == "EURUSD")
    )

    if not duplicate_condition.any():
        data_to_add = pd.DataFrame({
            'ID': [new_id],
            'Date': [last_date1],
            'Name': ['EURUSD'],
            'Price': [open_price1],
            'Quantity': [1]
        })
  
        combined_data = pd.concat([existing_data, data_to_add], ignore_index=True)

        combined_data.to_csv(csv_file_path, index=False)
        
        body = f'''Buy Alert, According to the model you should by EURUSD!'''
        send_mail(body)  
        x1=3

    existing_data = pd.read_csv(csv_file_path)    
if target_value2==1:
    new_id = len(existing_data) + 1
    duplicate_condition = (
        (existing_data["Date"] == last_date2) &
        (existing_data["Name"] == "GBPUSD")
    )

    if not duplicate_condition.any():
    
        data_to_add = pd.DataFrame({
            'ID': [new_id],
            'Date': last_date2,
            'Name': 'GBPUSD',
            'Price': open_price2,
            'Quantity': 1
        })
        combined_data = pd.concat([existing_data, data_to_add], ignore_index=True)

        combined_data.to_csv(csv_file_path, index=False)
        body = f'''Buy Alert, According to the model you should by GBPUSD!'''
        send_mail(body)
        x2=3
   



     
    existing_data = pd.read_csv(csv_file_path)    

if target_value3==1:
    new_id = len(existing_data) + 1
    duplicate_condition = (
        (existing_data["Date"] == last_date3) &
        (existing_data["Name"] == "USAU")
    )

    if not duplicate_condition.any():
    
        data_to_add = pd.DataFrame({
            'ID': [new_id],
            'Date': last_date3,
            'Name': 'USAU',
            'Price': open_price3,
            'Quantity': 1
        })
        combined_data = pd.concat([existing_data, data_to_add], ignore_index=True)

        combined_data.to_csv(csv_file_path, index=False)
        body = f'''Buy Alert, According to the model you should by USAU!'''
        send_mail(body)
        x3=3


     
    existing_data = pd.read_csv(csv_file_path)    

else:
    existing_data = pd.read_csv(csv_file_path)    
    

In [38]:
todayprice1=df1.tail(1)["close"].values[0]
todayprice2=df2.tail(1)["close"].values[0]
todayprice3=df3.tail(1)["close"].values[0]
today_value={
    "EURUSD": todayprice1,
    "GBPUSD": todayprice2,
    "USAU": todayprice3,
}
today_value

{'EURUSD': 1.057753324508667,
 'GBPUSD': 1.2172261476516724,
 'USAU': 3.4800000190734863}

Here it does the process of deleting item from the csv filke once that currencies is completely sold!

In [39]:
def delete_rows(passed_date, name):
    df=existing_data
    df = df[~((df['Date'] == passed_date) & (df['Name'] == name))]

    df = df.reset_index(drop=True)
    
    df['ID'] = range(1, len(df) + 1)
    return df

In [40]:
import pandas as pd


existing_data = pd.read_csv(csv_file_path)

for index, row in existing_data.iterrows():
    name = row['Name']
    p = row["Price"]

    if name == 'EURUSD':
        change_percentage = ((p - todayprice1) / todayprice1) * 100

    elif name == 'GBPUSD':
        change_percentage = ((p - todayprice2) / todayprice2) * 100

    elif name == 'USAU':
        change_percentage = ((p - todayprice3) / todayprice3) * 100

    z = 0
    if change_percentage >= 10:
        body = f'''Congrats, You have made more than 10% profit on {name}!
        Sell this stock! of date: {row["Date"]}'''
        existing_data = delete_rows(row["Date"], row["Date"])
        existing_data.to_csv(csv_file_path, index=False) 
    elif change_percentage >= 5:
        body = f'''Congrats, You have made more than 5% profit on {name}!
        Sell this stock, but only half! of date: {row["Name"]}'''
    elif change_percentage <= -5:
        body = f'''Sorry, You have experienced more than 5% loss on {name}!
        Sell this stock! of date: {row["Date"]}'''
        existing_data = delete_rows(row["Date"], row["Name"])
        existing_data.to_csv(csv_file_path, index=False) 
    
    elif change_percentage <= -2.5:
        body = f'''Sorry, You have experienced more than 2.5% loss on {name}!
        Sell this stock, but only half! of date: {row["Date"]}'''
    else:
        z = 1
        print(f"{name}: No significant change on {row['Name']}")

    if z == 0:
        send_mail(body)  




OUTPUT:


EURUSD: No significant change on EURUSD
GBPUSD: No significant change on GBPUSD
EURUSD: No significant change on EURUSD
GBPUSD: No significant change on GBPUSD
EURUSD: No significant change on EURUSD
GBPUSD: No significant change on GBPUSD

EMAIL:
to me

Sorry, You have experienced more than 5% loss on USAU!
        Sell this stock! of date: 2023-10-13

It was sent 3 time too.
 


If you need help with activating you email, please say so!
